After fine-tuning for many epochs with hyperparameter tuning, the performance increased from a ROUGE-1 score of 0.32 to 0.50.
**final evaluation**

In [2]:
from google.colab import drive#mounting google drive

drive.mount('/content/drive')

!pip install rouge-score #install library

#importing libraries
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
from rouge_score import rouge_scorer  #to calculate rough score

# Define a My dataset class
class MyDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=512, max_output_length=150):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.data.iloc[idx]['article']
        highlight = self.data.iloc[idx]['highlight']

        inputs = self.tokenizer.encode_plus(
            article,
            max_length=self.max_input_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        targets = self.tokenizer.encode_plus(
            highlight,
            max_length=self.max_output_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs.input_ids.flatten(),
            'attention_mask': inputs.attention_mask.flatten(),
            'labels': targets.input_ids.flatten()
        }

# Function to calculate ROUGE scores
def calculate_rouge_scores(hypotheses, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for hyp, ref in zip(hypotheses, references):
        scores = scorer.score(hyp, ref)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

    return avg_rouge1, avg_rouge2, avg_rougeL

# Load the validation data
val_data = pd.read_csv('/content/drive/My Drive/validate_data.csv')

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('/content/drive/My Drive/fine_tuning')
model = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/fine_tuning')

# Create the dataset and dataloader
val_dataset = MyDataset(val_data, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set model to evaluation mode
model.eval()
# Initialize empty lists to store generated summaries and target summaries
hypotheses = []
references = []
# Disable gradient calculation for inference
with torch.no_grad():
  # Iterate over batches in the validation data loader with progress bar
    for batch in tqdm(val_loader):
      # Move batch tensors to the appropriate device (GPU if available)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Generate summaries
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, num_beams=2, early_stopping=True)
         # Decode generated summaries and target summaries to text
        preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
        targets = [tokenizer.decode(l, skip_special_tokens=True, clean_up_tokenization_spaces=True) for l in labels]
 # Extend the hypotheses and references lists with the generated and target summaries
        hypotheses.extend(preds)
        references.extend(targets)

# Calculate ROUGE scores
rouge1, rouge2, rougeL = calculate_rouge_scores(hypotheses, references)

print(f'ROUGE-1: {rouge1:.4f} ROUGE-2: {rouge2:.4f} ROUGE-L: {rougeL:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=33b695c96107352288f6953dfee184e8ea39c7f130f7eebc26076df2f3c45c00
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 216/216 [06:21<00:00,  1.76s/it]


ROUGE-1: 0.5051 ROUGE-2: 0.3398 ROUGE-L: 0.4377


**example**

In [3]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load validation data (assuming 'val_data.csv' contains 'article' and 'highlight' columns)
val_data = pd.read_csv('/content/drive/My Drive/validate_data.csv')

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('/content/drive/My Drive/fine_tuning')
model = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/fine_tuning')

# Set device to GPU if available, otherwise to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate summaries
def generate_summary(article_text, tokenizer, model):
    inputs = tokenizer.encode_plus(
        article_text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    # Generate summary
    summary_ids = model.generate(
        inputs.input_ids.to(device),
        attention_mask=inputs.attention_mask.to(device),
        max_length=150,
        num_beams=2,
        early_stopping=True
    )

    # Decode the summary tokens to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return summary

# Select two records for demonstration
record1 = val_data.iloc[0]
record2 = val_data.iloc[1]

# Generate summaries
summary1 = generate_summary(record1['article'], tokenizer, model)
summary2 = generate_summary(record2['article'], tokenizer, model)

# Display the results
print("Record 1:")
print("Article:")
print(record1['article'])
print("\nHighlight:")
print(record1['highlight'])
print("\nGenerated Summary:")
print(summary1)

print("\n-------------------------------------------------\n")

print("Record 2:")
print("Article:")
print(record2['article'])
print("\nHighlight:")
print(record2['highlight'])
print("\nGenerated Summary:")
print(summary2)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Record 1:
Article:
brendan rodgers deliver weekly briefing media friday afternoon previewing showdown arsenal saturday discussing future raheem sterling young england star moved step closer anfield exit revealing tv interview ready sign new contract anfield sterling 20 offered new 100000aweek contract stay club admitted flattered interest arsenal follow press conference happens 2pm host commentator brendan rodgers brought close broadcast section press conference rest quotes embargoed tonight summary rodgers insisted raheem sterling going anywhere summer focus moment purely football manager said relaxed contract situation liverpool superpower world football dont sell best players admit however sterlings interview conducted without prior consent club took surprise concentration weeks back purely focus football help continue development place hes made great strides last couple years last time sat plays representatives intention raheem come made feelings clear hopefully continue focus foot